<a href="https://colab.research.google.com/github/yaolihua081/AIG_PC/blob/main/ReadEbooks_BERT_GPT_XLNET_Summary_MCQ_FAQ_08162021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://www.gutenberg.org/files/65997/65997-h/65997-h.htm
https://www.gutenberg.org/ebooks/65997
https://www.gutenberg.org/

#08/10/21, Read freee ebook, literature book
#Create summary ,Passage, question, MCQ with 4 choice.
#output summary
1. Read public book domain link, output the link. Difficult to get all links at one time, so I read a few pages and out put the links, later combine all the links into one file.

2. Read each book using Article, and separate by separators to create book chapters.
3. BERT, XLNET, GPT2, create summary for each chapter, use max_length=1000, num_sentences=40, and output results. 
4. Create summary with min_length=100, max_length=200, num_sentences=12, then, for the summary, create MCQ questions and output.
5. For the summary from 3., re create summary with min_lengh=100, max_length=200, num_sentemces=12 and output in csv files.
6. for files from 5. run program Passage2Question.ipynb, to create MCQ questions for each passage, with item weblink, book title, chapter information.
7. TO do. For the summary created in 3 or 5, classify topics with pre trained model from PC 3800 items.
8. ALLPublicDomainBook.txt: it contains 1019 book links.

#Restart Run Time

In [ ]:
#!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en
!pip install bert-extractive-summarizer --upgrade --force-reinstall
!pip install spacy==2.2.3 --upgrade --force-reinstall
!pip install -U nltk
!pip install -U pywsd
import nltk
nltk.download('stopwords')
##nltk.download('popular')
!pip install sentencepiece
#!pip install BigBirdTokenizer
!pip install sentence-transformers==0.2.5.1
!pip install transformers==2.6.0

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
from summarizer import Summarizer,TransformerSummarizer

bert_model = Summarizer()
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
XLNet_model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")

#Read an article on web

In [ ]:
!pip install newspaper3k
from newspaper import Article
from newspaper import Config
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 20


#Get URLs from a web site

In [ ]:
!pip install bs4
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup
 

#Get all public domain book links

In [ ]:
import re
urls = 'https://www.gutenberg.org/ebooks/search/?sort_order=downloads'
def BookList(url):
  grab = requests.get(url)
  soup = BeautifulSoup(grab.text, 'html.parser')
  bookList=[]
  for link in soup.find_all("a"):
    data = link.get('href')
    data='https://www.gutenberg.org'+str(data)
    bookList.append(data)
  return bookList

def GetBookList(urls,urllinks):
  bookList=BookList(urllinks)
  alllist=[]
  #f = open(fileout, "w")
  for i in range(len(bookList)):
    url=bookList[i]
    if urls not in url:
      ID_pattern=r'([\d]{1,6})'
      IDs = re.findall(ID_pattern, url)
      if (len(IDs)>0):
        num=int(IDs[0])
        link='https://www.gutenberg.org/files/'+str(num)+"/"+str(num)+"-h/"+str(num)+"-h.htm"
        alllist.append(link)
        #f.write(link)
        #f.write("\n")
  return alllist


def ContainBookList(urls,index,bookList):
  for j in range(len(bookList)):
    url=bookList[j]
    if urls in url:
      ID_pattern=r'([\d]{1,6})'
      idx= re.findall(ID_pattern, url)
      if  idx:  
        number=int(idx[0])
        if number >= index:
          return BookList(url), number 
  return 0, 0    
def AllBookList(urls,urllink,num, start=26):
  bookList=BookList(urllink)
  index=start
  alllist=GetBookList(urls,urllink)
  while num>0:
    #print(num)
    list, idx=ContainBookList(urls,index, bookList)
  
    if list !=0:
      bookList=list
      index=idx
      for i in range(len(bookList)):
        url=bookList[i]
        if urls not in url:
          ID_pattern=r'([\d]{1,6})'
          IDs = re.findall(ID_pattern, url)
          if (len(IDs)>0):
            tempnum=int(IDs[0])
            link='https://www.gutenberg.org/files/'+str(tempnum)+"/"+str(tempnum)+"-h/"+str(tempnum)+"-h.htm"
            alllist.append(link)
      num=num-1
  #f.close()    
  return alllist

#Get some pages of book link

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
number=30
fileout="/content/gdrive/My Drive/Result/PublicDomainBook30.txt"
urls = 'https://www.gutenberg.org/ebooks/search/?sort_order=downloads'
allbooklist=AllBookList(urls,number)

f = open(fileout, "w")
for i in allbooklist:
  f.write(str(i))
  f.write("\n")
f.close()  

In [ ]:
number=5
fileout="/content/gdrive/My Drive/Result/PublicDomainBook30-35.txt"
urls = 'https://www.gutenberg.org/ebooks/search/?sort_order=downloads'
urllink='https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index=751'
allbooklist=AllBookList(urls,urllink,number, start=751)

f = open(fileout, "w")
for i in allbooklist:
  f.write(str(i))
  f.write("\n")
f.close()  

#THis code combine all links into one file and out put

In [ ]:
filein="/content/gdrive/My Drive/Result/PublicDomainBook30.txt"
fileObject = open(filein, "r")
data = fileObject.read()
booklist1=data.split('\n') #776 link

filein="/content/gdrive/My Drive/Result/PublicDomainBook30-35.txt"
fileObject = open(filein, "r")
data = fileObject.read()
booklist2=data.split('\n') #776 link

filein="/content/gdrive/My Drive/Result/PublicDomainBook35-37.txt"
fileObject = open(filein, "r")
data = fileObject.read()
booklist3=data.split('\n')#776 link

filein="/content/gdrive/My Drive/Result/PublicDomainBook37-39.txt"
fileObject = open(filein, "r")
data = fileObject.read()
booklist4=data.split('\n')#776 link

booklist1.extend(booklist2)
booklist1.extend(booklist3)
booklist1.extend(booklist4)
allbooklist=list(set(booklist1))
booklist=[x for x in allbooklist if x !=""]
fileout="/content/gdrive/My Drive/Result/PublicDomainBook.txt"
f = open(fileout, "w")
for i in allbooklist:
  f.write(str(i))
  f.write("\n")
f.close()  

In [ ]:
fileout="/content/gdrive/My Drive/Result/AllPublicDomainBook.txt"
f = open(fileout, "w")
for i in allbooklist:
  f.write(str(i))
  f.write("\n")
f.close()  

In [ ]:
import re, numpy as np, pandas as pd
np.random.seed(400)



In [ ]:

fileout="/content/gdrive/My Drive/Result/PublicDomainBook.csv"
df=set(allbooklist)
pd.DataFrame(df).to_csv(fileout)

In [ ]:
from google.colab import files
files.upload()

Saving AllPublicDomainBook.txt to AllPublicDomainBook.txt


{'AllPublicDomainBook.txt': b'https://www.gutenberg.org/files/600/600-h/600-h.htm\nhttps://www.gutenberg.org/files/31284/31284-h/31284-h.htm\nhttps://www.gutenberg.org/files/383/383-h/383-h.htm\nhttps://www.gutenberg.org/files/1056/1056-h/1056-h.htm\nhttps://www.gutenberg.org/files/66033/66033-h/66033-h.htm\nhttps://www.gutenberg.org/files/28500/28500-h/28500-h.htm\nhttps://www.gutenberg.org/files/66109/66109-h/66109-h.htm\nhttps://www.gutenberg.org/files/66003/66003-h/66003-h.htm\nhttps://www.gutenberg.org/files/1497/1497-h/1497-h.htm\nhttps://www.gutenberg.org/files/20239/20239-h/20239-h.htm\nhttps://www.gutenberg.org/files/25063/25063-h/25063-h.htm\nhttps://www.gutenberg.org/files/3761/3761-h/3761-h.htm\nhttps://www.gutenberg.org/files/1155/1155-h/1155-h.htm\nhttps://www.gutenberg.org/files/66024/66024-h/66024-h.htm\nhttps://www.gutenberg.org/files/5998/5998-h/5998-h.htm\nhttps://www.gutenberg.org/files/18442/18442-h/18442-h.htm\nhttps://www.gutenberg.org/files/66011/66011-h/66011-h

In [ ]:
fileObject = open("AllPublicDomainBook.txt", "r")
data = fileObject.read()
allbooklist=data.split('\n')


In [ ]:
len(allbooklist) #there are 1019, 1018
type(allbooklist)
#allbooklist=[x for x in allbooklist if len(x)>0]
booklist1=allbooklist[:300]
booklist1
booklist2=allbooklist[301:600]
booklist3=allbooklist[601:]

In [ ]:
len(booklist1)

300

In [ ]:
all_tokens=split_string(article.text,"\n\n ")
for paragraph in all_tokens:
  print(paragraph)

In [ ]:
import time
config.request_timeout = 20
url=allbooklist[2]
#article=Article(url,config=config)
article=Article(url)
#article.set_html(url)
article.download()
while article.download_state ==0: #ArticleDownloadState.NOT_STARTED is 0
    time.sleep(1)
article.parse()
article.nlp
print(article.text)
#separators=['Chapter']
#all_tokens=split_string(article.text,separators)
#all_tokens


#08/19/21, there is a problem, artilce.text soes not contain all the things on the page /or the book. WHY? it was cut at somewhere.

In [ ]:
def split_string(source,separators): 
  return re.split('|'.join(re.escape(x) for x in separators),source)
             
def ReadBook(url,max_length,separators):
  article=Article(url)
  #article.download()
  try:
    article.download()
    article.parse() 
    #article.nlp() #this cause problem
    #all_tokens=re.split(r'Chapter|CHAPTER|\n\n', article.text)
    all_tokens=split_string(article.text,separators)
    all_tokens=[x  for x in all_tokens if len(x)>max_length]
    #all_tokens=all_tokens[15:]
    return all_tokens[1:],article.title 
  except:
    return 0,0
 

def CreateBertSummary(url,min_length, max_length , num_sentences,ratio,separators):
  dat,title=ReadBook(url,max_length,separators)
  if (dat !=0):
    result=[]
    for i in range(len(dat)):
      result.append(bert_model(dat[i],min_length=min_length, max_length=max_length , num_sentences=num_sentences, ratio=ratio))
    return result,title, url 
  else:
    return 0,0,0

def CreateXLNetSummary(url,min_length, max_length , num_sentences,ratio,separators):
  dat,title=ReadBook(url,max_length,separators)
  if (dat !=0):
    result=[]
    for i in range(len(dat)):
      result.append(XLNet_model(dat[i],min_length=min_length, max_length=max_length , num_sentences=num_sentences,ratio=ratio))
    return result,title, url 
  else:
    return 0,0,0   

def CreateGPT2Summary(url,min_length, max_length , num_sentences,ratio,separators):
  dat,title=ReadBook(url,max_length,separators)
  if (dat !=0):
    result=[]
    for i in range(len(dat)):
      result.append(GPT2_model(dat[i],min_length=min_length, max_length=max_length , num_sentences=num_sentences,ratio=ratio))
    return result,title, url   
  else:
    return 0,0,0          

#Create Sunmmary for books

In [ ]:
#XLNet_model(docfile, num_sentences=num_sentences)
def BertSummary(dat, min_length, max_length,num_sentences, ratio,fileout,separators):
  with open(fileout, 'w') as f:
    for i in range(len(dat)):
      url=dat[i]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateBertSummary(url,min_length, max_length , num_sentences,ratio,separators)
      print ("\nBook Title : ", title,"\n", file=f)
      if (result !=0):
        for j in range(len(result)):
          m=j+1
          print ("\nChapter : ", m,"\n", file=f)
          print ("Passage : ", result[j],file=f)
      
def GPT2Summary(dat, min_length, max_length,num_sentences, ratio,fileout,separators):
  with open(fileout, 'w') as f:
    
    for i in range(len(dat)):
      url=dat[i]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateGPT2Summary(url,min_length, max_length , num_sentences,ratio,separators)
      print ("\nBook Title : ", title,"\n", file=f)
      if (result !=0):
        for j in range(len(result)):
          m=j+1
          print ("\nChapter : ", m,"\n", file=f)
          print ("Passage : ", result[j],file=f)

def XLNetSummary(dat, min_length, max_length,num_sentences, ratio,fileout,separators):
  with open(fileout, 'w') as f:
    
    for i in range(len(dat)):
      url=dat[i]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateXLNetSummary(url,min_length, max_length , num_sentences,ratio,separators)
      print ("\nBook Title : ", title,"\n", file=f)
      if (result !=0):
        for j in range(len(result)):
          m=j+1
          print ("\nChapter : ", m,"\n", file=f)
          print ("Passage : ", result[j],file=f)        

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Create file conating summaries for each book
min_length=100, max_length=1000, num_sentences=40

# a minimum word count of about 100 words, and maximum of about 200 words, not to exceed 12 lines.

In [ ]:

num_sentences=40
summarizeratio=1
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/BertSummary"+str(summarizeratio)+".txt"
BertSummary(dat=allbooklist, min_length=100, max_length=1000, num_sentences=num_sentences, ratio=summarizeratio,fileout=fileout,separators=separators)

In [ ]:
booklist1

In [ ]:
num_sentences=40
summarizeratio=1
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']
fileout="/content/gdrive/My Drive/Result/Book1000BertSummary1"+str(summarizeratio)+".txt"
BertSummary(dat=booklist1, min_length=100, max_length=1000, num_sentences=num_sentences, ratio=summarizeratio,fileout=fileout,separators=separators)

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (11) found smaller than n_clusters (13). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
num_sentences=40
summarizeratio=1
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']
fileout="/content/gdrive/My Drive/Result/Book1000BertSummary2"+str(summarizeratio)+".txt"
BertSummary(dat=booklist2, min_length=100, max_length=1000, num_sentences=num_sentences, ratio=summarizeratio,fileout=fileout,separators=separators)

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (16). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
num_sentences=40
summarizeratio=1
#booklist3=allbooklist[601:]
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']
fileout="/content/gdrive/My Drive/Result/Book1000BertSummary3"+str(summarizeratio)+".txt"
BertSummary(dat=booklist3, min_length=100, max_length=1000, num_sentences=num_sentences, ratio=summarizeratio,fileout=fileout,separators=separators)

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
num_sentences=12
summarizeratio=1
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/BertSummaryLarge"+str(summarizeratio)+".txt"
BertSummary(dat=booklist, min_length=100, max_length=1000, num_sentences=25, ratio=summarizeratio,fileout=fileout,separators=separators)

In [ ]:
num_sentences=12
summarizeratio=1
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/XLNetSummaryLarge"+str(summarizeratio)+".txt"
XLNetSummary(dat=booklist, min_length=100, max_length=1000, num_sentences=40, ratio=summarizeratio,fileout=fileout,separators=separators)

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (16). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (16). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
num_sentences=25
summarizeratio=1
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/GTP2Summary"+str(summarizeratio)+".txt"
GPT2Summary(dat=booklist, min_length=100, max_length=500, num_sentences=num_sentences, ratio=summarizeratio,fileout=fileout,separators=separators)

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (16). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (16). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


#Re create short summary from larger summary and output results
# a minimum word count of about 100 words, and maximum of about 200 words, not to exceed 12 lines.

In [ ]:
def split_doc(filein,min_length):
  fileObject = open(filein, "r")
  dat = fileObject.read()
  df=pd.DataFrame()
  urls = re.findall('(?:(?:https?|ftp):\/\/)[\w/\-?=%.]+\.[\w/\-&?=%.]+', dat)
  #L=len(urls)
  data1=dat.split('Web URL')
  L=len(data1)
  for i in range(L):
    data2=data1[i].split('Chapter')
    title_link="Web URL"+str(data2[0])
    data3=[x for x in data2[1:] if len(x)>min_length]
    J=len(data3) #number of passages that will be used to create questions
    for j in range(J):
      text=data3[j].replace("Passage", " ")
      text=text.replace(':'," ")
      pattern=r'([\d]{1,2})'
      temp=re.findall(pattern,text) #chapter number
      text=text.replace(temp[0],"")
      text=text.replace('\n\n'," ")
      df=df.append(pd.Series([title_link,temp[0], text]),ignore_index=True)
  df.columns=['Book Web URL and Title','Chapter', 'Passage']
  return df
summarizeratio=1
filein="/content/gdrive/My Drive/Result/BertSummaryLarge"+str(summarizeratio)+".txt"
#df=split_doc(filein,min_length=100) 

In [ ]:
df

In [ ]:
def ShortBertSummary(text,min_length, max_length , num_sentences):
  result=bert_model(text,min_length=min_length, max_length=max_length , num_sentences=num_sentences)
  return result

def ShortXLNetSummary(text,min_length, max_length , num_sentences):
  result=XLNet_model(text,min_length=min_length, max_length=max_length , num_sentences=num_sentences)
  return result
def ShortGPT2Summary(text,min_length, max_length , num_sentences):
  result=GPT2_model(text,min_length=min_length, max_length=max_length , num_sentences=num_sentences)
  return result  


In [ ]:
summarizeratio=1
filein="/content/gdrive/My Drive/Result/Book1000BertSummaryLarge"+str(summarizeratio)+".txt"
df=split_doc(filein,min_length=100) 
max_length=200
num_sentences=12
min_length=100
df['BertSummary']=df['Passage'].apply(lambda x: ShortBertSummary(x,min_length, max_length , num_sentences))
pd.DataFrame(df).to_csv('PublicBook1000BertSummary.csv')

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
summarizeratio=1
filein="/content/gdrive/My Drive/Result/Book1000BertSummary1"+str(summarizeratio)+".txt"
df=split_doc(filein,min_length=100) 
max_length=200
num_sentences=12
min_length=100
df['BertSummary']=df['Passage'].apply(lambda x: ShortBertSummary(x,min_length, max_length , num_sentences))
pd.DataFrame(df).to_csv('PublicBook1000BertSummary1.csv')

In [ ]:
summarizeratio=1. 
filein="/content/gdrive/My Drive/Result/Book1000BertSummary2"+str(summarizeratio)+".txt"
df=split_doc(filein,min_length=100) 
max_length=200
num_sentences=12
min_length=100
df['BertSummary']=df['Passage'].apply(lambda x: ShortBertSummary(x,min_length, max_length , num_sentences))
pd.DataFrame(df).to_csv('PublicBook1000BertSummary2.csv')

In [ ]:
summarizeratio=1
filein="/content/gdrive/My Drive/Result/Book1000BertSummary3"+str(summarizeratio)+".txt"
df=split_doc(filein,min_length=100) 
max_length=200
num_sentences=12
min_length=100
df['BertSummary']=df['Passage'].apply(lambda x: ShortBertSummary(x,min_length, max_length , num_sentences))
pd.DataFrame(df).to_csv('PublicBook1000BertSummary3.csv')

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
df1=pd.read_csv('PublicBook1000BertSummary1.csv')
df2=pd.read_csv('PublicBook1000BertSummary2.csv')
df3=pd.read_csv('PublicBook1000BertSummary3.csv')
df=pd.concat([df1,df2,df3],axis=0)
fileout="/content/gdrive/My Drive/Result/PublicBook1000BertSummary.csv"
pd.DataFrame(df).to_csv(fileout)#there are 11654 rows

In [ ]:
len(df)

11654

In [ ]:
summarizeratio=1
filein="/content/gdrive/My Drive/Result/XLNetSummaryLarge"+str(summarizeratio)+".txt"
filein="/content/gdrive/My Drive/Result/Book1000BertSummaryLarge"+str(summarizeratio)+".txt"
df=split_doc(filein,min_length=100) 
max_length=200
num_sentences=12
min_length=100
df['XLNetSummary']=df['Passage'].apply(lambda x: ShortXLNetSummary(x,min_length, max_length , num_sentences))
pd.DataFrame(df).to_csv('PublicBook1000XLNetSummary.csv')

#Generate some MCQ questions

In [ ]:

import nltk
nltk.download('stopwords')
nltk.download('popular')
#python -m nltk.downloader 'popular'
!pip install sentencepiece
#!pip install BigBirdTokenizer
!pip install sentence-transformers==0.2.5.1
!pip install transformers==2.6.0
!python -m spacy download en

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('popular')
#python -m nltk.downloader 'popular'
!pip install sentencepiece
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize
!pip install flashtext
from flashtext import KeywordProcessor

#import requests
import json
import re
import random
from nltk.corpus import wordnet as wn

In [ ]:
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)] # list of list
    sentences = [y for x in sentences for y in x] #sentences[0], a list of sentences
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences
       
def get_nouns_multipartite(text,num=20, pos={'PROPN'}):#number of keys extracted
    out=[]

    extractor = pke.unsupervised.MultipartiteRank()
    #extractor = pke.unsupervised.TfIdf()
    extractor.load_document(input=text)
    #extractor.load_document(input=text,
    #                    language='en')

    #    not contain punctuation marks or stopwords as candidates.
    #pos={'PROPN'}
    #pos = {'VERB', 'ADJ', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos, stoplist=stoplist)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=num)

    for key in keyphrases:
        out.append(key[0])

    return out

# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    synsets = wn.synsets(word,'n')
    
    if synsets:
      try:
        wup = max_similarity(sent, word, pos='n')#this wup might not be the original word, may be root, cooling, cool
        try:
          adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
          if  wup in synsets and  adapted_lesk_output in synsets:
            lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
          elif wup in synsets:
            lowest_index=synsets.index(wup)
          elif adapted_lesk_output in synsets:  
            lowest_index=synsets.index(adapted_lesk_output)
          else:
            lowest_index=0
        except Exception:
          lowest_index=synsets.index(wup)
      except Exception:
        lowest_index=0
      return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

#create summary and create list of  key that was in the summary
summarizeratio=1
def CreatKeys(text, min_length,max_length,num_sentences,ratio,num, pos):
  keywords = get_nouns_multipartite(text,num=num, pos=pos) 
  result = bert_model(text, min_length=min_length, max_length = max_length , num_sentences=num_sentences,  ratio=ratio)
  summarized_text = ''.join(result)
  filtered_keys=[]
  for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
        filtered_keys.append(keyword)
  return filtered_keys ,summarized_text     


def Key_list(text,min_length,max_length,num_sentences,ratio,num=20, pos={'PROPN'}):
  key_distractor_list = {}
  filtered_keys,summarized_text = CreatKeys(text, min_length,max_length,num_sentences,ratio,num, pos)
  sentences = tokenize_sentences(summarized_text)
  keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
  for keyword in keyword_sentence_mapping:
    if len(keyword_sentence_mapping[keyword])>0:
      wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
      if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
          distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
          key_distractor_list[keyword] = distractors
      else:
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
          key_distractor_list[keyword] = distractors

  return key_distractor_list , keyword_sentence_mapping   

import sys  

def CreateBERTSummaryFormatMCQ(dat, min_length, max_length ,num_sentences,ratio,num, pos,separators,fileout):
  item=1
  with open(fileout, 'w') as f:
    for j in range(len(dat)):
      url=dat[j]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateBertSummary(url,min_length, max_length ,num_sentences, ratio,separators)
      print ("\nBook Title : ", title,"\n", file=f)
      for i in range(len(result)): 
        index=1
        text=result[i]
        i=i+1
        key_distractor_list,keyword_sentence_mapping=Key_list(text,min_length,max_length,num_sentences,ratio,num=num, pos=pos)
        print ("\nChapter : ", i,"\n", file=f)
        print ("\nPassage : ", text,"\n",file=f)
        for each in key_distractor_list:
          if len(keyword_sentence_mapping[each])>0: #modified 07/30/21 Lihua
            sentence = keyword_sentence_mapping[each][0]
            pattern = re.compile(each, re.IGNORECASE)
            output = pattern.sub( " _______ ", sentence)
            #print ("\nDocument Number : ", item,"\n",file=fileout)
            print ("%s)"%(index),output,file=f)
            choices = [each.capitalize()] + key_distractor_list[each]
            if len(choices)>=4:
              top4choices = choices[:4]
              random.shuffle(top4choices)
              optionchoices = ['a','b','c','d']
              for idx,choice in enumerate(top4choices):
                print ("\t",optionchoices[idx],")"," ",choice,file=f)
              print ("\nMore options: ", choices[4:20],"\n\n",file=f) 
              index=index+1     
     
def CreateGPT2SummaryFormatMCQ(dat, min_length, max_length ,num_sentences,ratio,num, pos,separators,fileout):
  item=1
  with open(fileout, 'w') as f:
    for j in range(len(dat)):
      url=dat[j]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateGPT2Summary(url,min_length, max_length ,num_sentences, ratio,separators)
      print ("\nBook Title : ", title,"\n", file=f)
      for i in range(len(result)): 
        index=1
        text=result[i]
        i=i+1
        key_distractor_list,keyword_sentence_mapping=Key_list(text,min_length,max_length,num_sentences,ratio,num=num, pos=pos)
        print ("\nChapter : ", i,"\n", file=f)
        print ("\nPassage : ", text,"\n",file=f)
        for each in key_distractor_list:
          if len(keyword_sentence_mapping[each])>0: #modified 07/30/21 Lihua
            sentence = keyword_sentence_mapping[each][0]
            pattern = re.compile(each, re.IGNORECASE)
            output = pattern.sub( " _______ ", sentence)
            #print ("\nDocument Number : ", item,"\n",file=fileout)
            print ("%s)"%(index),output,file=f)
            choices = [each.capitalize()] + key_distractor_list[each]
            if len(choices)>=4:
              top4choices = choices[:4]
              random.shuffle(top4choices)
              optionchoices = ['a','b','c','d']
              for idx,choice in enumerate(top4choices):
                print ("\t",optionchoices[idx],")"," ",choice,file=f)
              print ("\nMore options: ", choices[4:20],"\n\n",file=f) 
              index=index+1     


def CreateXLNetSummaryFormatMCQ(dat, min_length, max_length ,num_sentences,ratio,num, pos,separators,fileout):
  item=1
  with open(fileout, 'w') as f:
    for j in range(len(dat)):
      url=dat[j]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateXLNetSummary(url,min_length, max_length ,num_sentences, ratio,separators)
      print ("\nBook Title : ", title,"\n", file=f)
      for i in range(len(result)): 
        index=1
        text=result[i]
        i=i+1
        key_distractor_list,keyword_sentence_mapping=Key_list(text,min_length,max_length,num_sentences,ratio,num=num, pos=pos)
        print ("\nChapter : ", i,"\n", file=f)
        print ("\nPassage : ", text,"\n",file=f)
        for each in key_distractor_list:
          if len(keyword_sentence_mapping[each])>0: #modified 07/30/21 Lihua
            sentence = keyword_sentence_mapping[each][0]
            pattern = re.compile(each, re.IGNORECASE)
            output = pattern.sub( " _______ ", sentence)
            #print ("\nDocument Number : ", item,"\n",file=fileout)
            print ("%s)"%(index),output,file=f)
            choices = [each.capitalize()] + key_distractor_list[each]
            if len(choices)>=4:
              top4choices = choices[:4]
              random.shuffle(top4choices)
              optionchoices = ['a','b','c','d']
              for idx,choice in enumerate(top4choices):
                print ("\t",optionchoices[idx],")"," ",choice,file=f)
              print ("\nMore options: ", choices[4:20],"\n\n",file=f) 
              index=index+1                   


In [ ]:
dat=alllist[:2]
len(dat)
url=dat[0]
result, title, link=CreateBertSummary(url,min_length=100, max_length=200 ,num_sentences=12, ratio=0.2,separators=separators)

In [ ]:
summarizeratio=0.2
separators=['Chapter','CHAPTER','Letter','I','II','III''IV','V','VI','VII','VIII']
fileout="/content/gdrive/My Drive/Result/BERTSummaryMCQ"+str(summarizeratio)+".txt"
CreateBERTSummaryFormatMCQ(dat=allbooklist, min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,num=20, pos={'PROPN'},separators=separators,fileout=fileout)

In [ ]:
summarizeratio=0.2
separators=['Chapter','CHAPTER','Letter','I.','II.','III.''IV.','V.','VI.','VII.','VIII.']
for i in range(40):
  i=i+1
  ls1='Chapter '+ str(i)
  ls2='CHAPTER ' +str(i)
  ls2='Letter ' +str(i)
  separators.append(ls1)
  separators.append(ls2)
fileout="/content/gdrive/My Drive/Result/GPT2SummaryMCQ"+str(summarizeratio)+".txt"
CreateGPT2SummaryFormatMCQ(dat=alllist[:2], min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,num=20, pos={'PROPN'},separators=separators,fileout=fileout)

In [ ]:
summarizeratio=0.2
separators=['Chapter','CHAPTER','Letter','I.','II.','III.''IV.','V.','VI.','VII.','VIII.']
for i in range(40):
  i=i+1
  ls1='Chapter '+ str(i)
  ls2='CHAPTER ' +str(i)
  ls2='Letter ' +str(i)
  separators.append(ls1)
  separators.append(ls2)
fileout="/content/gdrive/My Drive/Result/XLNetSummaryMCQ"+str(summarizeratio)+".txt"
CreateXLNetSummaryFormatMCQ(dat=alllist[:2], min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,num=20, pos={'PROPN'},separators=separators,fileout=fileout)

#Another methods Creating MCQ items
1.Library

In [ ]:
!pip install git+https://github.com/ramsrigouthamg/Questgen.ai
!pip install sense2vec==1.0.2
!pip install git+https://github.com/boudinfl/pke.git

!python -m nltk.downloader universal_tagset
!python -m spacy download en

In [ ]:
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz
!ls s2v_old

#08/10/21. THis makes the code Questgen work, without restarting run time

In [ ]:
!pip install git+https://github.com/huggingface/transformers

#08/10/21. THis makes the code Questgen work, withoit restarting run time

In [ ]:
from pprint import pprint
import nltk
nltk.download('stopwords')
from Questgen import main
qe= main.BoolQGen()
qg = main.QGen()

In [ ]:
import random
def ReFormat(docfile):
  thisdict = {
     "input_text": docfile
  }
  return thisdict

def CreatePassageBert2MCQ(dat,min_length, max_length ,num_sentences, ratio,separators,fileout):
  
  with open(fileout, 'w') as f:
    for item in range(len(dat)):
      url=dat[item]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateBertSummary(url,min_length, max_length ,num_sentences, ratio,separators)
      if (result !=0):
        print ("\nBook Title : ", title,"\n", file=f)
        for i in range(len(result)): 
          index=0
          text=result[i]
          i=i+1
          tempdat=ReFormat(str(text))
          output = qg.predict_mcq(tempdat)
         # print ("\nChapter : ", i,"\n", file=f)
         # print ("\nPassage : ", text,"\n",file=f)
          if 'questions' in output.keys():
            L=len(output['questions'])
            for j in range(L):
              index=index+1
              answer, distractor, question=output['questions'][j]['answer'],output['questions'][j]['options'],output['questions'][j]['question_statement']
            
              #print ("%s)"%(index),question,file=f)
              distractor.append(answer.capitalize())
              if len(distractor)>=4:
                print ("\nChapter : ", i,"\n", file=f)
                print ("\nPassage : ", text,"\n",file=f)
                print ("%s)"%(index),question,file=f)
                random.shuffle(distractor)
                optionchoices = ['a','b','c','d']
                for idx,choice in enumerate(distractor):
                  print ("\t",optionchoices[idx],")"," ",choice,file=f)
            
        else:
          print ("None",file=f)
       
def CreatePassageXLNet2MCQ(dat,min_length, max_length ,num_sentences, ratio,separators,fileout):
  
  with open(fileout, 'w') as f:
    for item in range(len(dat)):
      url=dat[item]
      #print("j=",j)
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateXLNetSummary(url,min_length, max_length ,num_sentences, ratio,separators)
      if (result !=0):
        print ("\nBook Title : ", title,"\n", file=f)
        for i in range(len(result)): 
          index=0
          text=result[i]
          i=i+1

          tempdat=ReFormat(str(text))
          output = qg.predict_mcq(tempdat)
          #print ("\nChapter : ", i,"\n", file=f)
          #print ("\nPassage : ", text,"\n",file=f)
          if 'questions' in output.keys():
            L=len(output['questions'])
            for j in range(L):
              index=index+1
              #print("i=",i,"j=",j)
              answer, distractor, question=output['questions'][j]['answer'],output['questions'][j]['options'],output['questions'][j]['question_statement']
              distractor.append(answer.capitalize())
              if len(distractor)>=4:
                print ("\nChapter : ", i,"\n", file=f)
                print ("\nPassage : ", text,"\n",file=f)
                print ("%s)"%(index),question,file=f)
                random.shuffle(distractor)
                optionchoices = ['a','b','c','d']
                for idx,choice in enumerate(distractor):
                  print ("\t",optionchoices[idx],")"," ",choice,file=f)
              else:
                #print("NONE")
                print ("None",file=f)
      else:
        print ("None",file=f)

def CreatePassageGPT22MCQ(dat,min_length, max_length ,num_sentences, ratio,separators,fileout):
  
  with open(fileout, 'w') as f:
    for item in range(len(dat)):
      url=dat[item]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateGPT2Summary(url,min_length, max_length ,num_sentences, ratio,separators)
      if (result !=0):
        print ("\nBook Title : ", title,"\n", file=f)
        for i in range(len(result)): 
          index=0
          text=result[i]
          i=i+1
          tempdat=ReFormat(str(text))
          output = qg.predict_mcq(tempdat)
          #print ("\nChapter : ", i,"\n", file=f)
          #print ("\nPassage : ", text,"\n",file=f)
          if 'questions' in output.keys():
            L=len(output['questions'])
            for j in range(L):
              index=index+1
              answer, distractor, question=output['questions'][j]['answer'],output['questions'][j]['options'],output['questions'][j]['question_statement']
            
              #print ("%s)"%(index),question,file=f)
              distractor.append(answer.capitalize())
              if len(distractor)>=4:
                print ("\nChapter : ", i,"\n", file=f)
                print ("\nPassage : ", text,"\n",file=f)
                print ("%s)"%(index),question,file=f)
                random.shuffle(distractor)
                optionchoices = ['a','b','c','d']
                for idx,choice in enumerate(distractor):
                  print ("\t",optionchoices[idx],")"," ",choice,file=f)
              else:
                print ("None",file=f)
      else:
        print ("None",file=f)


In [ ]:
summarizeratio=0.5
separators=['Chapter','CHAPTER','ACT','STAVE','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/BERTSummary2MCQ3"+str(summarizeratio)+".txt"
CreatePassageBert2MCQ(dat=booklist3, min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,separators=separators,fileout=fileout)

In [ ]:
summarizeratio=0.5
separators=['Chapter','CHAPTER','ACT','STAVE','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/BERTSummary2MCQ2"+str(summarizeratio)+".txt"
CreatePassageBert2MCQ(dat=booklist2, min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,separators=separators,fileout=fileout)

In [ ]:
booklist1

In [ ]:
summarizeratio=0.5
separators=['Chapter','CHAPTER','ACT','STAVE','Letter','I','II','III''IV','V','VI','VII','VIII']

fileout="/content/gdrive/My Drive/Result/BERTSummary2MCQ1"+str(summarizeratio)+".txt"
CreatePassageBert2MCQ(dat=booklist1, min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,separators=separators,fileout=fileout)

In [ ]:
summarizeratio=0.5
separators=['Chapter','CHAPTER','ACT','STAVE','Letter','I','II','III''IV','V','VI','VII','VIII']


fileout="/content/gdrive/My Drive/Result/GPT2Summary2MCQ"+str(summarizeratio)+".txt"
CreatePassageGPT22MCQ(dat=booklist, min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,separators=separators,fileout=fileout)

In [ ]:
summarizeratio=0.2
separators=['Chapter','CHAPTER','ACT','STAVE','Letter','I','II','III''IV','V','VI','VII','VIII']
for i in range(40):
  i=i+1
  ls1='Chapter '+ str(i)
  ls2='CHAPTER ' +str(i)
  ls3='Letter ' +str(i)
  separators.append(ls1)
  separators.append(ls2)
  separators.append(ls3)
fileout="/content/gdrive/My Drive/Result/XLNetSummary2MCQ"+str(summarizeratio)+".txt"
CreatePassageXLNet2MCQ(dat=booklist, min_length=100, max_length=200 ,num_sentences=12,ratio=summarizeratio,separators=separators,fileout=fileout)

#Creat FAQ questions

In [ ]:
import random
def ReFormat(docfile):
  thisdict = {
     "input_text": docfile
  }
  return thisdict

def CreatePassageBert2FAQ(dat,min_length, max_length ,num_sentences, ratio,separators,fileout):
  with open(fileout, 'w') as f:
    for item in range(len(dat)):
      url=dat[item]
      print ("\nWeb URL : ", url,"\n", file=f)
      result, title, link=CreateBertSummary(url,min_length, max_length ,num_sentences, ratio,separators)
      if (result !=0):
        print ("\nBook Title : ", title,"\n", file=f)
        for i in range(len(result)): 
          index=0
          text=result[i]
          i=i+1
          tempdat=ReFormat(str(text))
          output = qg.predict_shortq(tempdat)
          if 'questions' in output.keys():
            L=len(output['questions'])
            for j in range(L):
              index=index+1
              answer, question=output['questions'][j]['Answer'],output['questions'][j]['Question']
              print ("\nChapter : ", i,"\n", file=f)
              print ("\nPassage : ", text,"\n",file=f)
              print ("%s)"%(index),question,file=f)
            
        else:
          print ("None",file=f)
       

      else:
        print ("None",file=f)



#above all work

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
summarizeratio=0.5
CreateFormatMCQ(ls, ratio=summarizeratio,num=10, pos={'PROPN'},fileout='/content/gdrive/My Drive/Result/Book1MCQ0.5.txt')

In [ ]:
summarizeratio=0.1
CreateMCQ(ls[0], ratio=summarizeratio,num=10, pos={'PROPN'},fileout='/content/gdrive/My Drive/Result/BookMCQ_Ratio0.1.txt')

In [ ]:
import os
import zipfile

bert_wsd_pytorch = "/content/gdrive/My Drive/ReadPDF/Automate_the_Boring_Stuff_2e_onlinematerials.zip"
extract_directory = "/content/gdrive/My Drive/ReadPDF"


extracted_folder = bert_wsd_pytorch.replace(".zip","")

#  If unzipped folder exists don't unzip again.
if not os.path.isdir(extracted_folder):
  with zipfile.ZipFile(bert_wsd_pytorch, 'r') as zip_ref:
      zip_ref.extractall(extract_directory)
else:
  print (extracted_folder," is extracted already")

In [ ]:
import PyPDF2

import PyPDF2
pdfFileObj = open('/content/gdrive/My Drive/ReadPDF/automate_online-materials/meetingminutes.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pdfReader.numPages
num=1
pageObj = pdfReader.getPage(num)
text=pageObj.extractText()

In [ ]:
import re, numpy as np, pandas as pd
np.random.seed(400)
#LDA_gensm_dominant_topics_modelLRG_5_allSummary=pd.read_csv('/content/gdrive/My Drive/Result/Bert_MCQ.csv')
df=pd.read_csv('Passage_FAQ.csv')
df=df.drop(labels=['Unnamed: 0'],axis=1)

In [ ]:
PdfText = PdfText.replace("\n", " ")
print(PdfText)

In [ ]:
!pip install PyPDF2